**Baseline classifier model. Op basis van TF-IDF Logistische regressie. Moet class imbalance nog aanpakken.**

In [1]:
""""
wat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"
daarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"
"""


'"\nwat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"\ndaarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"\n'

In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd


import os

In [3]:
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemes.xlsx")
df = pd.read_excel(file_path)

# 2. Data verkennen
print("Dataset overzicht:")
print(df.info())
print(df.head())

Dataset overzicht:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92503 entries, 0 to 92502
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   context      92010 non-null  object
 1   question     92503 non-null  object
 2   statistical  92503 non-null  int64 
 3   theme        92503 non-null  object
 4   file_name    92503 non-null  object
 5   clean_text   92503 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.2+ MB
None
                                             context  \
0  Ondertussen is de eerstelijnszone BruZEL al me...   
1  Ondertussen is de eerstelijnszone BruZEL al me...   
2  Ondertussen is de eerstelijnszone BruZEL al me...   
3  Ondertussen is de eerstelijnszone BruZEL al me...   
4  Ondertussen is de eerstelijnszone BruZEL al me...   

                                            question  statistical  \
0  Zoals alle eerstelijnszones kreeg ook BruZEL h...            0   
1              

In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
print(f"Number of rows after filtering: {len(df)}")
# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df["question"])
y = df["theme"]

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



print(f"Original dataset size: {X_train.shape[0]}")


Number of rows after filtering: 92503
Original dataset size: 74002


In [6]:
# ✅ Hyperparameter tuning met GridSearchCV
# Conditional grid - alleen 'elasticnet' krijgt l1_ratio
param_grid = {
    'C': [0.1, 1, 10],  # Minder waarden testen
    'penalty': ['l1', 'l2'],  # ElasticNet weglaten als dat niet veel verschil maakt
    'solver': ['saga']
}

grid = GridSearchCV(
    LogisticRegression(max_iter=500),  # Model
    param_grid,  # Zoekruimte
    cv=5,  # 5-fold cross-validation
    scoring="f1_weighted",  # Optimaliseer voor F1-score
    n_jobs=-1,  # Parallel berekenen
    verbose=2  # ✅ Geeft gedetailleerde logs per iteratie
)

print("GridSearchCV wordt uitgevoerd... Dit kan even duren 🚀")
grid.fit(X_train, y_train)  # GridSearch starten

# ✅ Print beste parameters en scores
print(f"Beste parameters: {grid.best_params_}")
best_model = grid.best_estimator_

# ✅ Evaluatie op de testset
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

# ✅ Compute Accuracy, Precision, Recall, and F1-score (same format as BERT output)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=1)

# ✅ Format Output as DataFrame
baseline_results = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1"],
    "Score": [accuracy, precision, recall, f1]
})

# ✅ Display results in a table format
print(baseline_results)


GridSearchCV wordt uitgevoerd... Dit kan even duren 🚀
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Beste parameters: {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
                              precision    recall  f1-score   support

                   Begroting       0.78      0.58      0.66       272
           Bestuur en Beleid       0.67      0.61      0.64      1147
  Brussel en de Vlaamse Rand       0.76      0.41      0.53       165
     Cultuur en Communicatie       0.63      0.53      0.58       629
          Economie en Arbeid       0.65      0.65      0.65      1575
                     Energie       0.67      0.53      0.59       621
                   Financiën       0.77      0.59      0.67       233
       Internationaal Beleid       0.73      0.56      0.63       390
      Justitie en Handhaving       0.72      0.43      0.54       260
          Milieu en Landbouw       0.60      0.66      0.63      2906
Mobiliteit en Infrastructuur       0.74      0.84   